In [1]:
from transformers import AutoTokenizer
from transformers import RobertaTokenizer, EncoderDecoderModel
import pandas as pd
import gradio as gr


model = EncoderDecoderModel.from_pretrained("models/rob2rand_chen_w_prefix_26-5-22/checkpoint-70000")
tokenizer = RobertaTokenizer.from_pretrained("models/rob2rand_chen_w_prefix_26-5-22/checkpoint-70000")

def generate_preds(desc, gen_mode, num_beams, num_returned_seqs):
    desc = desc.lower()
    if gen_mode=="Channel":
        desc = "GENERATE TRIGGER AND ACTION CHANNEL ONLY <pf> " + desc 
    elif gen_mode=="Function":
        desc = "GENERATE BOTH CHANNEL AND FUNCTION FOR TRIGGER AND ACTION <pf> " + desc 
        
    input_ids = tokenizer.encode(desc, return_tensors='pt')
    
    # activate beam search and early_stopping
    preds = model.generate(
        input_ids, 
        max_length=100, 
        num_beams=num_beams,
        num_return_sequences=num_returned_seqs,
        early_stopping=True
    )
    
    output_list = []
    for item in preds:
        output_list.append(tokenizer.decode(item, skip_special_tokens=True))
    
    if gen_mode=="Channel":
        trigger = [x.split("<sep>")[0].strip() for x in output_list]
        trigger_desc = ["dummy" for x in output_list]
        action = [x.split("<sep>")[1].strip() for x in output_list]
        action_desc = ["dummy" for x in output_list]
        df = {"Trigger": trigger,
              "Trigger Description": trigger_desc,
              "Action": action,
              "Action Description": action_desc
             }
    elif gen_mode=="Function":
        trigger = [x.split("<sep>")[1].strip() for x in output_list]
        trigger_desc = ["dummy" for x in output_list]
        action = [x.split("<sep>")[3].strip() for x in output_list]
        action_desc = ["dummy" for x in output_list]
        df = {"Trigger": trigger,
              "Trigger Description": trigger_desc,
              "Action": action,
              "Action Description": action_desc
             }
    return pd.DataFrame(df)


def generate_preds_field(desc_field, num_beams_field, num_returned_seqs_field):
    desc_field = desc_field.lower()
    
    input_ids = tokenizer.encode(desc_field, return_tensors='pt')
    
    # activate beam search and early_stopping
    preds = model.generate(
        input_ids, 
        max_length=100, 
        num_beams=num_beams_field,
        num_return_sequences=num_returned_seqs_field,
        early_stopping=True
    )
    
    output_list = []
    for item in preds:
        output_list.append(tokenizer.decode(item, skip_special_tokens=True))
    
    trigger = [x.split("<sep>")[0].strip() for x in output_list]
    trigger_desc = ["dummy" for x in output_list]
    trigger_fields = ["dummy" for x in output_list]
    action = [x.split("<sep>")[1].strip() for x in output_list]
    action_desc = ["dummy" for x in output_list]
    action_fields = ["dummy" for x in output_list]
    df = {"Trigger": trigger,
          "Trigger Description": trigger_desc,
          "Trigger Fields": trigger_fields,
          "Action": action,
          "Action Description": action_desc,
          "Action Fields": action_fields
         }
    return pd.DataFrame(df)


demo = gr.Blocks()
with demo:
    gr.Markdown("<h1><center>RecipeGen: Automated Trigger Action Programs (TAPs) Generation Tool</center></h1>")
    # gr.Markdown("This demo allows you to generate TAPs using functionality description described in English. You can learn the working detail of our tool from our paper")
    gr.Markdown("<h3>What is TAP?</h3>")
    gr.Markdown("""
        TAPs or Trigger Action Programs are an event-driven rules used to automate smart devices and/or internet services. TAPs are written in the form of "IF <trigger> then execute <action>, 
        where <trigger> and <action> refer to API calls. TAPs has been used in various use cases, ranging from home monitoring system to business workflow automation.
        """)
    gr.Markdown("<h3>Why generating TAPs can be challenging?</h3>")
    gr.Markdown("""
        Generating TAP is challenging mainly because of the large search space of the API calls that can be used as the trigger and action.
    """)
    gr.Markdown("<h3>RecipeGen</h3?>")
    gr.Markdown("""
        RecipeGen is a deep learning-based tool that can assist end-users to generate TAPs using natural language description. End-users can describe the functionality of the intended TAP, then RecipeGen
        will generate the candidate TAPs based those descriptions.
    """)
    gr.Markdown("<h3>Instructions</h3>")
    gr.Markdown("""
    1. Select the generation granularity (i.e., Channel, Function, or Field)
    2. Describe your intended functionality 
    3. Specify the beam width
    4. Specify the number of returned sequences
    5. Click **Generate**; the generated TAPs along with the description of each component will show in the **Results**
    """)
    gr.Markdown("NOTE: **#Returned Sequences** should be LESS THAN OR EQUAL **Beam Width**")
    with gr.Tabs():
        with gr.TabItem("Channel/Function"):    
            with gr.Column():
                gen_mode = gr.Radio(label="Granularity", choices=["Channel", "Function"])
                desc = gr.Textbox(label="Functionality Description", placeholder="Describe the functionality here")
                num_beams = gr.Slider(minimum=2, maximum=500, value=10, step=1, label="Beam Width")
                num_returned_seqs = gr.Slider(minimum=2, maximum=500, value=10, step=1, label="#Returned Sequences")
                generate = gr.Button("Generate")
                
            with gr.Box():
                gr.Markdown("<h3><center>Results</center></h3>")
                gr.Markdown("<br>")
                results = gr.Dataframe(headers=["Trigger", "Trigger Description", "Action", "Action Description"])
        
        with gr.TabItem("Field"):    
            with gr.Column():
                desc_field = gr.Textbox(label="Functionality Description", placeholder="Describe the functionality here")
                num_beams_field = gr.Slider(minimum=2, maximum=500, value=10, step=1, label="Beam Width")
                num_returned_seqs_field = gr.Slider(minimum=2, maximum=500, value=10, step=1, label="#Returned Sequences")
                generate_field = gr.Button("Generate")
                
            with gr.Box():
                gr.Markdown("<h3><center>Results</center></h3>")
                gr.Markdown("<br>")
                results_field = gr.Dataframe(headers=["Trigger", "Trigger Description", "Trigger Fields", "Action", "Action Description", "Action Fields"])
    
    generate.click(generate_preds, inputs=[desc, gen_mode, num_beams, num_returned_seqs], outputs=[results])
    generate_field.click(generate_preds_field, inputs=[desc_field, num_beams_field, num_returned_seqs_field], outputs=[results_field])
demo.launch(server_port=8333)

Running on local URL:  http://localhost:8333/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f655bcb74c0>, 'http://localhost:8333/', None)

In [2]:
# generate_preds_field("send me a notification when someone tag me in facebook", 2, 2)